In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/huggingface-tf2-roberta-base-class-weights/__results__.html
/kaggle/input/huggingface-tf2-roberta-base-class-weights/__resultx__.html
/kaggle/input/huggingface-tf2-roberta-base-class-weights/__notebook__.ipynb
/kaggle/input/huggingface-tf2-roberta-base-class-weights/__output__.json
/kaggle/input/huggingface-tf2-roberta-base-class-weights/new_submission.csv
/kaggle/input/huggingface-tf2-roberta-base-class-weights/custom.css
/kaggle/input/roberta-base/rust_model.ot
/kaggle/input/roberta-base/config.json
/kaggle/input/roberta-base/merges.txt
/kaggle/input/roberta-base/README.md
/kaggle/input/roberta-base/tokenizer.json
/kaggle/input/roberta-base/vocab.json
/kaggle/input/roberta-base/tf_model.h5
/kaggle/input/roberta-base/dict.txt
/kaggle/input/roberta-base/pytorch_model.bin
/kaggle/input/roberta-base/flax_model.msgpack
/kaggle/input/twitter-airline-sentiment/Tweets.csv
/kaggle/input/twitter-airline-sentiment/database.sqlite


In [7]:
import pandas as pd


The pandas package is imported using the variable pd

In [8]:
data = pd.read_csv('/kaggle/input/twitter-airline-sentiment/Tweets.csv')

In [9]:
print(data)

                 tweet_id airline_sentiment  airline_sentiment_confidence  \
0      570306133677760513           neutral                        1.0000   
1      570301130888122368          positive                        0.3486   
2      570301083672813571           neutral                        0.6837   
3      570301031407624196          negative                        1.0000   
4      570300817074462722          negative                        1.0000   
...                   ...               ...                           ...   
14635  569587686496825344          positive                        0.3487   
14636  569587371693355008          negative                        1.0000   
14637  569587242672398336           neutral                        1.0000   
14638  569587188687634433          negative                        1.0000   
14639  569587140490866689           neutral                        0.6771   

               negativereason  negativereason_confidence         airline  \

In [10]:
data.shape

(14640, 15)

The data size is 14640 x 15

In [11]:
data.isnull().sum()

tweet_id                            0
airline_sentiment                   0
airline_sentiment_confidence        0
negativereason                   5462
negativereason_confidence        4118
airline                             0
airline_sentiment_gold          14600
name                                0
negativereason_gold             14608
retweet_count                       0
text                                0
tweet_coord                     13621
tweet_created                       0
tweet_location                   4733
user_timezone                    4820
dtype: int64

We now identify the columns that have blank responses.

In [12]:
data["airline"].drop_duplicates()

0        Virgin America
504              United
4326          Southwest
6746              Delta
8966         US Airways
11879          American
Name: airline, dtype: object

VIRGIN AMERICA, UNITED, SOUTHWEST, DELTA, US AIRWAYS, AMERICAN are the airlines.

In [13]:
data["airline_sentiment_confidence"].drop_duplicates()

0        1.0000
1        0.3486
2        0.6837
6        0.6745
7        0.6340
          ...  
14562    0.7257
14569    0.7241
14587    0.6384
14594    0.7094
14635    0.3487
Name: airline_sentiment_confidence, Length: 1023, dtype: float64

In [14]:
data["airline_sentiment"].drop_duplicates()

0     neutral
1    positive
3    negative
Name: airline_sentiment, dtype: object

PHASE 2 BEGINS

In [15]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

**PHASE 3,4 - DEVELOPMENT**

In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.preprocessing import LabelEncoder

# Load the dataset
dataset_path = '/kaggle/input/twitter-airline-sentiment/Tweets.csv'
df = pd.read_csv(dataset_path)

# Select relevant columns
df = df[['text', 'airline_sentiment']]

# Split the dataset into training, validation, and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(test_df, test_size=0.5, random_state=42)

# Text preprocessing
# You can customize this part to suit your specific data cleaning needs.
# Common preprocessing steps include lowercasing, removing special characters, and tokenization.

# Vectorize the text data using Count Vectorization and TF-IDF transformation
count_vectorizer = CountVectorizer()
tfidf_transformer = TfidfTransformer()

X_train_counts = count_vectorizer.fit_transform(train_df['text'])
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

X_val_counts = count_vectorizer.transform(val_df['text'])
X_val_tfidf = tfidf_transformer.transform(X_val_counts)

X_test_counts = count_vectorizer.transform(test_df['text'])
X_test_tfidf = tfidf_transformer.transform(X_test_counts)

# Encode the sentiment labels
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(train_df['airline_sentiment'])
y_val = label_encoder.transform(val_df['airline_sentiment'])
y_test = label_encoder.transform(test_df['airline_sentiment'])


In [17]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW, get_linear_schedule_with_warmup
import torch
from torch.utils.data import DataLoader, TensorDataset, RandomSampler, SequentialSampler
from tqdm import tqdm

# Load and preprocess the dataset (as mentioned in your previous code)

# Initialize RoBERTa tokenizer and model from your locally downloaded 'roberta-base'
tokenizer = RobertaTokenizer.from_pretrained("/kaggle/input/roberta-base")
model = RobertaForSequenceClassification.from_pretrained("/kaggle/input/roberta-base", num_labels=3)  # 3 labels: positive, negative, neutral

# Tokenize and encode the text data
X_train_encoded = tokenizer(train_df['text'].tolist(), padding=True, truncation=True, return_tensors="pt")
X_val_encoded = tokenizer(val_df['text'].tolist(), padding=True, truncation=True, return_tensors="pt")

# Convert labels to PyTorch tensors
y_train_tensor = torch.tensor(y_train)
y_val_tensor = torch.tensor(y_val)

# Create data loaders
train_data = TensorDataset(X_train_encoded.input_ids, X_train_encoded.attention_mask, y_train_tensor)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=32)

val_data = TensorDataset(X_val_encoded.input_ids, X_val_encoded.attention_mask, y_val_tensor)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=64)

# Set up training parameters
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_dataloader) * 2)

# Training loop (as in the previous code)

# Now, you can use the trained model for sentiment analysis


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /kaggle/input/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [19]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification

# Specify the path to your local directory containing the "roberta-base" model
local_model_path = "/kaggle/input/roberta-base"

# Load the RoBERTa model and tokenizer from the local directory
model = RobertaForSequenceClassification.from_pretrained(local_model_path, num_labels=3)
tokenizer = RobertaTokenizer.from_pretrained(local_model_path)

# Rest of your code for testing the model remains the same


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /kaggle/input/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification

# Load the pre-trained RoBERTa model and tokenizer
model = RobertaForSequenceClassification.from_pretrained("/kaggle/input/roberta-base", num_labels=3)
tokenizer = RobertaTokenizer.from_pretrained("/kaggle/input/roberta-base")

# Define the tweet
tweet = "such an agonizing experience caused by these extraordinary so-called airlines"

# Tokenize and encode the tweet
encoded_tweet = tokenizer(tweet, padding=True, truncation=True, return_tensors="pt")

# Make predictions
with torch.no_grad():
    model.eval()
    outputs = model(**encoded_tweet)
    predictions = torch.argmax(outputs.logits, dim=1)

# Map predictions to labels
sentiment_labels = ['negative', 'neutral', 'positive']
predicted_sentiment = sentiment_labels[predictions.item()]

print("The sentiment of the tweet is: ",{predicted_sentiment})


In [ ]:
import torch
from transformers import RobertaForSequenceClassification, RobertaTokenizer, TrainingArguments, Trainer
from datasets import load_dataset

# Load the pre-trained RoBERTa model and tokenizer
tokenizer = RobertaTokenizer.from_pretrained("/kaggle/input/roberta-base")
model = RobertaForSequenceClassification.from_pretrained("/kaggle/input/roberta-base")

# Load your dataset (assuming you have it in a format compatible with the 'datasets' library)
dataset = load_dataset('/kaggle/input/twitter-airline-sentiment')

# Tokenize and preprocess your dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="steps",
    save_steps=500,
    eval_steps=500,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    evaluation_strategy="steps",
    save_total_limit=2,
    load_best_model_at_end=True,
    remove_unused_columns=False,
    push_to_hub=False,
)

# Define a data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Start fine-tuning
trainer.train()

# Save the model
trainer.save_model('./your_fine_tuned_model')
